## RBF para reconhecimento de padrão:

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter, LogLocator

import numpy as np
import pandas as pd
import os

import random
from math import cos, tanh
from time import sleep

from ipywidgets import *
import ipywidgets as widgets
import re

from IPython.display import display, HTML

########## entradas dos parametros ##########
style = {'description_width': 'initial'}

print('\n\nTaxa de aprendizagem:')
taxa_aprendizagem = widgets.FloatSlider(
    value=0.01,
    min=0.01,
    max=1,
    step=0.01,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
display(taxa_aprendizagem)

print('\n\nQuantidade de neurônios da camada intermediária:')
qnt_neuronios = widgets.IntSlider(
    value=5,
    min=1,
    max=100,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(qnt_neuronios)

print('\n\nErro tolerado:')
erro = widgets.FloatSlider(
    value=0.06,min=0,max=1,step=0.001,
    continuous_update=False,
    readout=True, style=style,
    readout_format='.5f',
)
display(erro)


class RBF():
    def __init__(self, conjunto_treinamento, targets_treinamento, taxa_aprendizagem, parada, ciclos_max, 
                 erro_tolerado, qnt_neuronios, **kwargs):
        self.numero_entradas = 256 #tamanho da imagem
        self.numero_saidas = 16 #0-9
        self.conjunto_treinamento = np.array(conjunto_treinamento)
        self.targets_treinamento = np.array(targets_treinamento)
        self.taxa_aprendizagem = taxa_aprendizagem
        self.ciclos_treinamento = 0
        self.ciclos_max = ciclos_max #Caso seja realizado treinamento por ciclos
        self.parada = parada #condições de parada
        self.qnt_neuronios_camada_intermediaria = qnt_neuronios
        self.erro_tolerado = erro_tolerado
        self.pesos_vij = self.__inicializa_pesos_camada_intermediaria__() #Inicializa pesos da camada intermediária
        self.pesos_wij = self.__inicializa_pesos_camada_saida__() #Inicializa pesos da camada de saída
        self.bias_saida = np.array([random.uniform(-0.5,0.5) for i in range(self.numero_saidas)]) #Inicializa os bias da camada de
        self.erro = 0    
   
    def __inicializa_pesos_camada_intermediaria__(self):
        pesos_vij = []
        for i in range(self.qnt_neuronios_camada_intermediaria):
            pesos_vij.append(self.conjunto_treinamento[i])
        return np.array(pesos_vij)
    
    def __inicializa_pesos_camada_saida__(self):
        pesos_wij = []
        #nxk onde n é o número de neurônios da camada intermediária e k é o número de saídas
        for i in range(self.qnt_neuronios_camada_intermediaria):
            peso_aux = []
            for i in range(self.numero_saidas):
                peso_aux.append(random.uniform(-0.5,0.5))
            pesos_wij.append(peso_aux)
            
        return np.array(pesos_wij)
    
    
    def treinamento(self):
        #Definir as amostras de treinamento
        #Definir o array de pesos
        
        #######Etapa 1 do treinamento
        print(np.sum(np.power((self.pesos_vij[0] - self.conjunto_treinamento[1]),2)))
        calculo = np.power(np.sum(np.power((self.pesos_vij[0] - self.conjunto_treinamento[1]),2)), 0.5)
        print(calculo)
        treinada = False
        #Inicializando array de amostras
        amostras = []
        amostras_anterior = []
        ciclo = 0
        #Inicializando a lista de amostras
        for i in range(self.qnt_neuronios_camada_intermediaria):
            amostras.append([])
        
        #Passo 3
        while not treinada:
            #Passo 3.1
            for entrada in self.conjunto_treinamento:
                distancias = []
                for peso_vij in self.pesos_vij:
                    distancia = np.power(np.sum(np.power((peso_vij - entrada),2)), 0.5)
                    distancias.append(distancia)
                menor_distancia = np.amin(distancias)
                j = distancias.index(menor_distancia)
                #i, = np.where(distancias == menor_distancia)
                amostras[j].append(entrada)
                
            #Passo 3.2
            for i in range(len(self.pesos_vij)):
                for j in range(len(amostras[i])):
                    if j == 0:
                        aux_somatorio = amostras[i][j]
                    else: 
                        aux_somatorio += amostras[i][j]
                
                self.pesos_vij[i] = aux_somatorio / len(amostras[i])
            if amostras == amostras_anterior:
                treinada = True
            else:
                amostras_anterior = amostras
        
        #Passo 4
        variancias = []
        for i in range(len(self.pesos_vij)):
            variancia = 0
            for j in range(len(amostras[i])):
                variancia = np.sum(np.power((amostras[i][j] - self. pesos_vij), 2))
            variancias.append(variancia / len(amostras[i]))
        ####### Fim da etapa 1 do treinamento
        
        #Etapa 2 do treinamento
        #1 Obtendo o conjunto de entradas da camada de saída (Já obtido na criação da classe)
        #2 Obter o vetor de saídas desejadas (já obtido na criação da classe)
        #3 inicializar os pesos (já obtido na criação da classe)
        #4 Especificar taxa de aprendizageme  erro (já obtidos na criação da classe)
        #5 Calcular entradas para a camada de saída
        novo_conjunto_entradas = []
        for entrada in self.conjunto_treinamento:
            nova_entrada = []
            for i in range(len(self.pesos_vij)):
                exp = np.sum(np.power((entrada - self.pesos_vij[i]), 2))/(2 * variancias[i])
                nova_entrada.append(np.exp(-exp))
            novo_conjunto_entradas.append(nova_entrada)
        novo_conjunto_entradas = np.array(novo_conjunto_entradas)
        
        treinada = False
        while not treinada:
            print('fim')
            return
        
       
            
            
                
                    

##### Lendo o arquivo de entradas da rede digitostreinamento900.txt #####
def get_inputs_from_txt():
    inputs = []
    file = open('digitos/digitostreinamento900.txt', 'r')
    for line in file :
        line = line.split(" ")
        ipt = []
        for element in line:
            if element != "":
                ipt.append(float(element))
        inputs.append(ipt)
    file.close()
    return np.array(inputs)
##############################################

##### Lendo os arquivos 450 arquivos de teste da rede#####
def get_inputs_test_from_txt():
    path_to_folder = 'C:\\Users\\Marco Antônio\\Documents\\IA\\Trabalhos\\trabalhos-ia\\trab10\\teste'
    inputs = []
    for i in os.listdir(path_to_folder):
        file = open(path_to_folder + '\\' + i, 'r')
        for line in file :
            line = line.split(" ")
            ipt = []
            for element in line:
                if element != "":
                    ipt.append(float(element))
            inputs.append(ipt)
        file.close()
    return np.array(inputs)
##############################################

def get_targets():
    targets_aux = pd.read_csv('targets.csv').values
    targets = []
    for i in range(90):
        for j in range(10):
            targets.append(list(targets_aux[j]))
    return np.array(targets)


def processar_RBF():
    conjunto_treinamento = get_inputs_from_txt()
    targets_treinamento = get_targets()
    rbf = RBF(
        conjunto_treinamento = conjunto_treinamento, 
        targets_treinamento = targets_treinamento, 
        taxa_aprendizagem = 0.01,
        parada = 'erro',
        ciclos_max = 0,
        erro_tolerado = erro.value,
        qnt_neuronios = qnt_neuronios.value
    )
    
    rbf.treinamento()
    
widgets.interact_manual.opts['manual_name'] = 'Treinar a rede' # muda texto do botao
interact_manual(processar_RBF); # metodo a executar quando pressionar o botao



Taxa de aprendizagem:


FloatSlider(value=0.01, continuous_update=False, max=1.0, min=0.01, readout_format='.3f', step=0.01)



Quantidade de neurônios da camada intermediária:


IntSlider(value=5, continuous_update=False, min=1)



Erro tolerado:


FloatSlider(value=0.06, continuous_update=False, max=1.0, readout_format='.5f', step=0.001, style=SliderStyle(…

interactive(children=(Button(description='Treinar a rede', style=ButtonStyle()), Output()), _dom_classes=('wid…

In [23]:
import numpy as np

test = [[1,2,3],[2,5,6]]
test1 = [[1,2,3],[2,5,6]]
np.array(test)
np.array(test1)
print(test == test1)

True
